In [68]:
import requests
import pandas as pd
import io
from datetime import datetime

pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes sans tronquer
pd.set_option('display.max_rows', None)  

In [42]:




def get_data(url,path_file,url_2,columns):
    # Utilisez la fonction requests.get() pour récupérer le contenu de l'URL
    response = requests.get(url)
    # Vérifiez que la requête a réussi
    response.raise_for_status()
    # Lisez le contenu de la réponse sous forme de CSV et convertissez-le en DataFrame
    df_nb_veh = pd.read_json(io.StringIO(response.text), encoding='utf-8')

    # information departement
    df_dep = pd.read_csv(path_file,dtype={"code_departement": str}, encoding='utf-8')


    # Utilisez la fonction requests.get() pour récupérer le contenu de l'URL
    response_2 = requests.get(url_2)

    # Vérifiez que la requête a réussi
    response_2.raise_for_status()

    # Lisez le contenu de la réponse sous forme de CSV et convertissez-le en DataFrame
    df_chrg = pd.read_csv(io.StringIO(response_2.text),low_memory=False
                    ,parse_dates=["date_mise_en_service"]
                    ,dtype={"consolidated_code_postal": str
                        }, encoding='utf-8')\
    .sort_values(['id_pdc_itinerance', 'last_modified'])
    df_chrg = df_chrg[columns]
    
    return df_nb_veh, df_dep, df_chrg


In [69]:

def process_func(df_chrg):

    # Définir la date d'aujourd'hui
    today = pd.Timestamp.today().normalize()

    df_process = df_chrg.copy()
    # On ne peut pas avoir une puissance >2Mega Watt pour un point de charge 

    df_process.puissance_nominale = df_chrg.puissance_nominale.apply(
        lambda x: x if x < 2000 else x / 1000
    )

    df_process = df_process[(df_process['date_mise_en_service'] >= '2015-01-01') &
                             (df_process['date_mise_en_service'] <= today)]  

    df_process = df_process.sort_values(["id_pdc_itinerance", "last_modified"])

    # id_pdc_itinerance represente l'ID du point de charge qui est supposé etre unique
    df_process = df_process.drop_duplicates("id_pdc_itinerance", keep="last")

    # nouvelle colonne departement
    df_process["code_departement"] = df_process["consolidated_code_postal"].str[:2]
    return df_process



In [44]:
def agg_func(df):
    # Créer un nouveau DataFrame avec l'agrégation par codgeo
    df_agg = df.groupby('codgeo')['nb_vp_rechargeables_el'].sum().reset_index()
    print(df_agg.head())

    return df_agg



In [45]:
def join_func(df_chrg,df_dep,df_agg):
    df_chrg = pd.merge(df_chrg,df_dep, how='left', on='code_departement')
    df_join = df_chrg.merge(df_agg, how='left', left_on='code_insee_commune', right_on='codgeo')

    return df_join

In [46]:
def pourc_col_na(df):
    # Calcul du pourcentage de valeurs nulles pour chaque colonne
    null_percentages = (df.isnull().mean() * 100).round(0)

    # Affichage des pourcentages de valeurs nulles pour chaque colonne
    print("Pourcentage de valeurs nulles par colonne :")
    print(null_percentages)




In [47]:
def drop_col(df,column_na):
    # Supprimer les lignes avec NaN dans la colonne "code_departement"
    df_non_na = df.dropna(subset=column_na)
    return df_non_na

In [48]:
def write_func(df):
    # Enregistrer le DataFrame résultant dans un fichier CSV
    df.to_csv('data/donnees_traitees.csv', index=False)

In [49]:
url_nb_veh = "https://www.data.gouv.fr/fr/datasets/r/9d0b1476-62bd-4612-8279-98d9d25959c1"
path_file = "C:/Users/khaled/Mon espace de travail/E/master 2 ynov/deep learning/projet_final/data/dep/departements-france.csv"
url_chrg = "https://www.data.gouv.fr/fr/datasets/r/eb76d20a-8501-400e-b336-d85724de5435"


columns_filter= [
    'id_station_itinerance', 
    'id_pdc_itinerance',
    'nom_station', 
    'implantation_station',
    'code_insee_commune', 
    'coordonneesXY', 
    'nbre_pdc',
    'puissance_nominale', 
    'date_mise_en_service', 
    'date_maj', 
    'last_modified',
    'consolidated_longitude', 
    'consolidated_latitude',
    'consolidated_code_postal', 
    'consolidated_commune',
    'consolidated_is_lon_lat_correct',
    'consolidated_is_code_insee_verified'
]

df_nb_veh, df_dep, df_nb_chrg= get_data(url_nb_veh,path_file, url_chrg, columns_filter)

In [50]:
df_nb_veh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457348 entries, 0 to 457347
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   codgeo                   457348 non-null  object
 1   libgeo                   457348 non-null  object
 2   epci                     454465 non-null  object
 3   libepci                  454465 non-null  object
 4   date_arrete              457348 non-null  object
 5   nb_vp_rechargeables_el   457348 non-null  int64 
 6   nb_vp_rechargeables_gaz  457348 non-null  int64 
 7   nb_vp                    457348 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 27.9+ MB


In [51]:
df_nb_veh_freq = agg_func(df_nb_veh)


  codgeo  nb_vp_rechargeables_el
0  01001                     124
1  01002                      32
2  01004                    2914
3  01005                     302
4  01006                       1


In [52]:
df_nb_veh_freq["codgeo"].value_counts().sum()

35191

In [53]:
df_nb_veh_freq.describe()

,nb_vp_rechargeables_el
count,35191.000000
mean,337.870762
std,1873.268256
min,0.000000
25%,18.000000
50%,56.000000
75%,175.000000
max,112501.000000


In [54]:
df_dep.head()

,code_departement,nom_departement,code_region,nom_region
0,01,Ain,84,Auvergne-Rhône-Alpes
1,02,Aisne,32,Hauts-de-France
2,03,Allier,84,Auvergne-Rhône-Alpes
3,04,Alpes-de-Haute-Provence,93,Provence-Alpes-Côte d'Azur
4,05,Hautes-Alpes,93,Provence-Alpes-Côte d'Azur


In [55]:
pourc_col_na(df_dep)

Pourcentage de valeurs nulles par colonne :
code_departement    0.0
nom_departement     0.0
code_region         0.0
nom_region          0.0
dtype: float64


In [56]:
df_nb_chrg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97695 entries, 0 to 97610
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   id_station_itinerance                97695 non-null  object        
 1   id_pdc_itinerance                    97695 non-null  object        
 2   nom_station                          97695 non-null  object        
 3   implantation_station                 97695 non-null  object        
 4   code_insee_commune                   57227 non-null  object        
 5   coordonneesXY                        97695 non-null  object        
 6   nbre_pdc                             97695 non-null  int64         
 7   puissance_nominale                   97695 non-null  float64       
 8   date_mise_en_service                 64532 non-null  datetime64[ns]
 9   date_maj                             97695 non-null  object        
 10  last_modif

In [70]:
df_chrg_process = process_func(df_nb_chrg)

In [71]:
df_chrg_process.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62603 entries, 18 to 97613
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   id_station_itinerance                62603 non-null  object        
 1   id_pdc_itinerance                    62603 non-null  object        
 2   nom_station                          62603 non-null  object        
 3   implantation_station                 62603 non-null  object        
 4   code_insee_commune                   47674 non-null  object        
 5   coordonneesXY                        62603 non-null  object        
 6   nbre_pdc                             62603 non-null  int64         
 7   puissance_nominale                   62603 non-null  float64       
 8   date_mise_en_service                 62603 non-null  datetime64[ns]
 9   date_maj                             62603 non-null  object        
 10  last_modi

In [72]:
pourc_col_na(df_chrg_process)

Pourcentage de valeurs nulles par colonne :
id_station_itinerance                   0.0
id_pdc_itinerance                       0.0
nom_station                             0.0
implantation_station                    0.0
code_insee_commune                     24.0
coordonneesXY                           0.0
nbre_pdc                                0.0
puissance_nominale                      0.0
date_mise_en_service                    0.0
date_maj                                0.0
last_modified                           0.0
consolidated_longitude                  0.0
consolidated_latitude                   0.0
consolidated_code_postal               43.0
consolidated_commune                   33.0
consolidated_is_lon_lat_correct         0.0
consolidated_is_code_insee_verified     0.0
code_departement                       43.0
dtype: float64


In [73]:
df_join = join_func(df_chrg_process, df_dep, df_nb_veh_freq)  
df_join.describe()

,nbre_pdc,puissance_nominale,consolidated_longitude,consolidated_latitude,code_region,nb_vp_rechargeables_el
count,62603.000000,62603.000000,62603.000000,62603.000000,35832.000000,47277.000000
mean,11.244876,46.534495,2.680458,46.929353,48.827361,11903.774605
std,53.870646,78.672005,2.800824,2.610775,28.306164,22128.848892
min,1.000000,0.000000,-61.720480,-20.949535,11.000000,0.000000
25%,1.000000,7.360000,1.349890,44.853540,27.000000,543.000000
50%,2.000000,22.000000,2.393800,47.507438,52.000000,2607.000000
75%,6.000000,22.080000,4.794226,48.866081,76.000000,11296.000000
max,505.000000,400.000000,55.517910,61.520355,93.000000,112501.000000


In [74]:
pourc_col_na(df_join)

Pourcentage de valeurs nulles par colonne :
id_station_itinerance                   0.0
id_pdc_itinerance                       0.0
nom_station                             0.0
implantation_station                    0.0
code_insee_commune                     24.0
coordonneesXY                           0.0
nbre_pdc                                0.0
puissance_nominale                      0.0
date_mise_en_service                    0.0
date_maj                                0.0
last_modified                           0.0
consolidated_longitude                  0.0
consolidated_latitude                   0.0
consolidated_code_postal               43.0
consolidated_commune                   33.0
consolidated_is_lon_lat_correct         0.0
consolidated_is_code_insee_verified     0.0
code_departement                       43.0
nom_departement                        43.0
code_region                            43.0
nom_region                             43.0
codgeo                          

In [75]:

column=  "code_departement"

df_final = drop_col(df_join,column)
df_final.head()

,id_station_itinerance,id_pdc_itinerance,nom_station,implantation_station,code_insee_commune,coordonneesXY,nbre_pdc,puissance_nominale,date_mise_en_service,date_maj,last_modified,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified,code_departement,nom_departement,code_region,nom_region,codgeo,nb_vp_rechargeables_el
0,FR000011062174,FR000011062174,Hotel saint alban,Parking privÃ© Ã usage public,34199,"[3.407609123225763, 43.41959147913006]",1,22.0,2022-03-02,2022-04-25,2024-01-19T07:47:00.381000+00:00,3.407609,43.419591,34120,PÃ©zenas,False,True,34,Hérault,76.0,Occitanie,34199,1648.0
1,FR000012292701,FR000012292701,HÃ´tel Restaurant Campanile Nogent-sur-Marne,Parking privÃ© Ã usage public,94052,"[2.493569567590577, 48.832677935169805]",2,22.0,2022-02-22,2022-05-12,2024-01-19T07:47:00.381000+00:00,2.493570,48.832678,94130,Nogent-sur-Marne,True,True,94,Val-de-Marne,11.0,Île-de-France,94052,4845.0
3,FR000012616553,FR000012616553,1PACTE,Parking privÃ© Ã usage public,13041,"[5.476711409891, 43.476583984941]",2,22.0,2022-04-04,2022-05-04,2024-01-19T07:47:00.381000+00:00,5.476711,43.476584,13120,Gardanne,True,True,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur,13041,5325.0
4,FR000028067822,FR000028067822,Carry-le-Rouet,Parking privÃ© Ã usage public,13021,"[5.143766265497639, 43.3292004491334]",2,36.0,2023-01-13,2023-06-06,2024-01-19T07:48:04.429000+00:00,5.143766,43.329200,13620,Carry-le-Rouet,True,True,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur,13021,2882.0
5,FR026PYZERON,FR026E169008,YZERON,Parking privÃ© Ã usage public,69269,"[4.578864, 45.704218]",1,11.0,2023-07-06,2023-07-06,2024-01-19T07:49:07.361000+00:00,4.578864,45.704218,69510,Yzeron,True,True,69,Rhône,84.0,Auvergne-Rhône-Alpes,69269,165.0


In [79]:
df_final.columns

Index(['id_station_itinerance', 'id_pdc_itinerance', 'nom_station',
       'implantation_station', 'code_insee_commune', 'coordonneesXY',
       'nbre_pdc', 'puissance_nominale', 'date_mise_en_service', 'date_maj',
       'last_modified', 'consolidated_longitude', 'consolidated_latitude',
       'consolidated_code_postal', 'consolidated_commune',
       'consolidated_is_lon_lat_correct',
       'consolidated_is_code_insee_verified', 'code_departement',
       'nom_departement', 'code_region', 'nom_region', 'codgeo',
       'nb_vp_rechargeables_el'],
      dtype='object')

In [76]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35973 entries, 0 to 62602
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   id_station_itinerance                35973 non-null  object        
 1   id_pdc_itinerance                    35973 non-null  object        
 2   nom_station                          35973 non-null  object        
 3   implantation_station                 35973 non-null  object        
 4   code_insee_commune                   35973 non-null  object        
 5   coordonneesXY                        35973 non-null  object        
 6   nbre_pdc                             35973 non-null  int64         
 7   puissance_nominale                   35973 non-null  float64       
 8   date_mise_en_service                 35973 non-null  datetime64[ns]
 9   date_maj                             35973 non-null  object        
 10  last_modif

In [77]:
pourc_col_na(df_final)

Pourcentage de valeurs nulles par colonne :
id_station_itinerance                  0.0
id_pdc_itinerance                      0.0
nom_station                            0.0
implantation_station                   0.0
code_insee_commune                     0.0
coordonneesXY                          0.0
nbre_pdc                               0.0
puissance_nominale                     0.0
date_mise_en_service                   0.0
date_maj                               0.0
last_modified                          0.0
consolidated_longitude                 0.0
consolidated_latitude                  0.0
consolidated_code_postal               0.0
consolidated_commune                   0.0
consolidated_is_lon_lat_correct        0.0
consolidated_is_code_insee_verified    0.0
code_departement                       0.0
nom_departement                        0.0
code_region                            0.0
nom_region                             0.0
codgeo                                 0.0
nb_vp_rech

In [78]:
write_func(df_final)